# Notebook for backtesting the Machine Learning models

### In this notebook we will be using the saved joblib files, and using them to generate Buy / Dont buy signals for the data fetched from CCXT

The test data will be for the past 720 datapoints for each of 'BTC/AUD', 'ETH/AUD', 'XRP/AUD', 'LTC/AUD', 'ADA/AUD', 'XLM/AUD' and 'BCH/AUD'

In [1]:
# Import libraries and dependencies
import ccxt
import os
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import talib as ta

from Utility_Functions import Functions

# Control parameters for the backtest

**indicators_list** has to be the same as the indicators that were used while training the ML Model

In [2]:
model_for_testing = '2021-08-28_ada_boost.joblib'
curr_list = ['BTC/AUD', 'ETH/AUD', 'XRP/AUD', 'LTC/AUD', 'ADA/AUD', 'XLM/AUD', 'BCH/AUD']
indicators_list = ['BBands_high', 'BBands_low', 'RSI_ratio', 'CCI','ADX', 'ADX_dirn', 'SMA_vol_agg', 'MACD_ratio']

# Redownload the data just to check the time taken for each activity

In [3]:
# Load environment variables
load_dotenv()

# Import environment variables
kraken_public_key = os.getenv("KRAKEN_PUBLIC_KEY")
kraken_secret_key = os.getenv("KRAKEN_SECRET_KEY")

# Verify that environment variables were loaded
print(f"Kraken key data type: {type(kraken_public_key)}")
print(f"Kraken secren data type: {type(kraken_secret_key)}")

Kraken key data type: <class 'str'>
Kraken secren data type: <class 'str'>


### Set Public and private keys and initialise the CCXT Exchange object

In [4]:
# Set the public and private keys for the API
exchange = ccxt.kraken({
    'apiKey': kraken_public_key,
    'secret': kraken_secret_key,
})

if not exchange.has['fetchOHLCV']:
    print('Cant load history data with this exchange. Try with a different exchange')

### Fetching the price data

In [5]:
currs_list = curr_list     #

dict_ohlcv = {}

for curr in currs_list:
    
    # Call data fetch
    ohlcv = exchange.fetchOHLCV(curr, '1h')

    # Store the values in a dataframe
    df_ohlcv = pd.DataFrame(ohlcv, columns=['Date', 'Open', 'High', 'Low', 'Close', 'Volume']).set_index('Date')
    df_ohlcv.index = pd.to_datetime(df_ohlcv.index, unit='ms')

    df_ohlcv.dropna(inplace=True)

    # Store the symbol name and history data in a dict 
    dict_ohlcv[curr] = df_ohlcv 

    start_date = df_ohlcv.index[0].date().isoformat()
    end_date = df_ohlcv.index[-1].date().isoformat()
    num_records = (len(df_ohlcv))
    start_price = df_ohlcv.iloc[0]['Close']
    end_price = df_ohlcv.iloc[-1]['Close']

    print(f'Data summary for {curr}')
    print(f'    Start Date: {start_date}; End Date: {end_date}; NUmber of records: {num_records}')
    print(f'    Start Price: {start_price}; End Price: {end_price}')    
    print(f'Data for {curr} fetched and appended into the dictionary\n')

Data summary for BTC/AUD
    Start Date: 2021-07-30; End Date: 2021-08-29; NUmber of records: 720
    Start Price: 54195.2; End Price: 68000.0
Data for BTC/AUD fetched and appended into the dictionary

Data summary for ETH/AUD
    Start Date: 2021-07-30; End Date: 2021-08-29; NUmber of records: 720
    Start Price: 3286.35; End Price: 4510.57
Data for ETH/AUD fetched and appended into the dictionary

Data summary for XRP/AUD
    Start Date: 2021-07-30; End Date: 2021-08-29; NUmber of records: 720
    Start Price: 1.0215; End Price: 1.5833
Data for XRP/AUD fetched and appended into the dictionary

Data summary for LTC/AUD
    Start Date: 2021-07-30; End Date: 2021-08-29; NUmber of records: 720
    Start Price: 186.86; End Price: 240.36
Data for LTC/AUD fetched and appended into the dictionary

Data summary for ADA/AUD
    Start Date: 2021-07-30; End Date: 2021-08-29; NUmber of records: 720
    Start Price: 1.77703; End Price: 3.93754
Data for ADA/AUD fetched and appended into the dictio

### Computing and adding Technical Indicators to the price data

In [6]:
fast_window = 5
slow_window = 15

df_all_data = pd.DataFrame() 
for curr, data in dict_ohlcv.items():

    df = Functions.add_tech_indicators(data, fast_window, slow_window)
    
    df['Currency'] = curr 
    df['Returns'] = df['Close'].pct_change()

    df.dropna(inplace=True)    
    df_all_data = df_all_data.append(df)

df_all_data.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'SMA_agg', 'RSI_ratio', 'CCI',
       'MACD_ratio', 'ADX', 'ADX_dirn', 'ATR_ratio', 'BBands_high',
       'BBands_low', 'SMA_vol_agg', 'Currency', 'Returns'],
      dtype='object')

# Add the target values to check the performance of the strategy

In [7]:
df_all_data['Target_returns'] = df_all_data.Returns.shift(-1)
df_all_data.dropna(inplace=True)
df_all_data['Buy_or_sell'] = df_all_data.Target_returns.apply(lambda x: 1 if x > 0 else -1)
df_all_data.shape

(4934, 19)

# Backtesting this data

### Fetch the saved joblib file

In [8]:
from sklearn.pipeline import make_pipeline, Pipeline
from joblib import dump, load
from pathlib import Path

In [9]:
filename = 'Joblibs/' + model_for_testing
pipeline = load(filename)

### Start of predictions

In [31]:
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

In [ ]:
for curr_tested in curr_list:

    df_testing_subset = df_all_data.loc[ df_all_data.Currency == curr_tested].copy()
    df_testing_subset.sort_index(inplace=True)

    X_test = df_testing_subset.loc[: , indicators_list].reset_index(drop=True)   
    y_test = df_testing_subset.loc[:, ['Target_returns', 'Buy_or_sell']].copy()
    # y_test.head()

    # Run the predictions
    df_pred = y_test
    df_pred['Pred_buy_or_sell'] = pipeline.predict(X_test)
    # df_pred.head()

    print(f'\nClassification report for {curr_tested}')
    print(classification_report(y_test.Buy_or_sell, df_pred.Pred_buy_or_sell))

    df_all_buys = df_pred.loc[ df_pred.Pred_buy_or_sell == 1 ].copy()

    cum_rets = (1 + (df_all_buys['Target_returns'] * df_all_buys['Pred_buy_or_sell'])).cumprod()

    title = 'Cumulative returns for ' + curr_tested
    png_file = 'Backtest_results/Results_' + curr_tested.replace('/','-') + '.png'

    fig, ax = plt.subplots()
    ax.plot(cum_rets)

    ax.set(xlabel='Date', ylabel='Returns',
        title= title)
    ax.grid()

    fig.savefig(png_file)
